In [1]:
import os
os.popen('ls').read()

'PW002044.owl\nSMP0000060\nconvert_pathbank.ipynb\npathbank_all_biopax.zip\n'

In [27]:
def xml_key(l):
    if l and l[0] == '<' and l[1] != '/' and l[1] != '?':
        return l.split()[0][1:]

with open('PW002044.owl') as owl:
    owl = [l.rstrip() for l in owl.readlines()]
xml = set([xml_key(l) for l in owl if xml_key(l)])
print(xml)

{'owl:Ontology', 'bp:UnificationXref', 'bp:Complex', 'bp:ChemicalStructure', 'bp:Interaction', 'bp:ProteinReference', 'bp:Pathway', 'rdf:RDF', 'bp:CellularLocationVocabulary', 'bp:SmallMoleculeReference', 'bp:BiochemicalReaction', 'bp:Transport', 'bp:BiochemicalPathwayStep', 'bp:Protein', 'bp:Stoichiometry', 'bp:SmallMolecule', 'bp:BioSource', 'bp:PathwayStep', 'bp:Catalysis'}


['<?xml version="1.0" encoding="UTF-8"?>',
 '<rdf:RDF',
 ' xmlns="http://smpdb.ca/pathways/#"',
 ' xmlns:xsd="http://www.w3.org/2001/XMLSchema#"',
 ' xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"',
 ' xmlns:owl="http://www.w3.org/2002/07/owl#"',
 ' xmlns:bp="http://www.biopax.org/release/biopax-level3.owl#"',
 ' xml:base="http://smpdb.ca/pathways/#">',
 '<owl:Ontology rdf:about="">',
 ' <owl:imports rdf:resource="http://www.biopax.org/release/biopax-level3.owl#" />',
 '</owl:Ontology>',
 '',
 '<bp:UnificationXref rdf:ID="Reference/PubChem-compound_6022">',
 ' <bp:id rdf:datatype = "http://www.w3.org/2001/XMLSchema#string">6022</bp:id>',
 ' <bp:db rdf:datatype = "http://www.w3.org/2001/XMLSchema#string">PubChem-compound</bp:db>',
 '</bp:UnificationXref>',
 '',
 '<bp:SmallMoleculeReference rdf:ID="CompoundReference/PW_C000940">',
 ' <bp:xref rdf:resource="#Reference/BioCyc_ACETYL-COA" />',
 ' <bp:xref rdf:resource="#Reference/PubChem-compound_444493" />',
 ' <bp:xref rdf:resour